In [1]:
import pickle
from flask import Flask, json, jsonify, make_response, request
from flask_cors import CORS

In [2]:
file = open('vectorizer.pickle', 'rb')
vectorizer = pickle.load(file)
file.close() 

file = open('lr.pickle', 'rb')
model_lr = pickle.load(file)
file.close()

file = open('nb.pickle', 'rb')
model_nb = pickle.load(file)
file.close()

file = open('svm.pickle', 'rb')
model_svm = pickle.load(file)
file.close()

def get_result(model, text_vector, text):
    if(model == 'lr'):
        return logistic_regression(text_vector, text)
    if(model == 'nb'):
        return naive_bayes(text_vector, text)
    if(model == 'svm'):
        return svm(text_vector, text)
    
    return jsonify({'error':'sorry! unable to load model', 'status_code':500})

def logistic_regression(text_vector, text):
    result = model_lr.predict(text_vector)
    return json.dumps({'sentiment': int(result[0]), 'text': text, 'model':'Logistic Regression'})

def naive_bayes(text_vector, text):
    result = model_nb.predict(text_vector)
    return json.dumps({'sentiment': int(result[0]), 'text': text, 'model':'Naive Bayes'})

def svm(text_vector, text):
    result = model_svm.predict(text_vector)
    return json.dumps({'sentiment': int(result[0]), 'text': text, 'model':'Support Vector Machine'})

In [3]:
app = Flask(__name__)
app.config['JSON_AS_ASCII'] = False
CORS(app)

@app.route('/sentiment', methods=['GET'])
def sentiment_analysis():
    if request.method == 'GET':
        text = request.args.get('text')
        model = request.args.get('model')

        if text:
            text_vector = vectorizer.transform([text])
            result = get_result(model, text_vector, text)
            return make_response(result, 200)
            
        return make_response(jsonify({'error':'sorry! unable to parse', 'status_code':500}), 500)

if __name__ == '__main__':
   app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Apr/2022 18:54:14] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2022 18:54:18] "GET /sentiment?model=lr&text=stupid%20nerd HTTP/1.1" 200 -
